# Notebook for creating regex functions to plagiate c code

In [1]:
import re

def remove_comments(given_string, regexs = ["\n?[ |\t]*//[\t  \S]*", "\n?[ |\t]*/\*[\s\S]*?\*/"]):
    for regex in regexs :
        all_matches = re.findall(regex,given_string)
        for match in all_matches:
            given_string = given_string.replace(match, "")
    return given_string

## Working with dataframes

In [2]:
import pandas as pd
import numpy as np
from obfuscator import variable_renamer
from csv_stuff import create_labled_table_routine, add_valid_code_columns, remove_given_code
import sys
# sys.path.append('/home/erik/TU/ni/Source-Code-Plagiarism-Detection/Source codes/')
sys.path.append('../../../Source-Code-Plagiarism-Detection/Source codes/')
# TODO use init file
from remove_given_code_from_student_solution import remove_given_code_from_df


PATH_TEMPLATE = "/home/erik/TU/ni/plagiate_labeltool/data/code_templates/"


def create_plagiate_table(df, semester, ha, task, prog_language, number_labled_pairs, path):
    df_labled = pd.DataFrame(columns=['semester', 'ha', 'task', 'prog_lang',
                                      'surname1', 'lastname1', 'code1', 'code2', 'label'])

    df = add_valid_code_columns(df, semester, ha, [task], prog_language)
    number_of_valid_codes = len(df.loc[df[f'{task} empty'] == 0])
    print("number_of_valid_codes: " + str(number_of_valid_codes))
    plagiate_per_solution = int(np.floor(number_labled_pairs/number_of_valid_codes))
    print("plagiate_per_solution: " + str(plagiate_per_solution))
    for ln, sn, code in df.loc[df[f'{task} empty'] == 0][['Nachname', 'Vorname', task]].values:
        for i in range(plagiate_per_solution):
            plagiate_code = remove_comments(code)
            plagiate_code = variable_renamer(plagiate_code)
            df_labled.loc[len(df_labled)] = [
                semester, ha, task, prog_language, sn, ln, code, plagiate_code, 1]
    df_labled.to_csv(path)
    return df_labled


def create_plagiate_table_routine(semester, ha, task, prog_language):
    csv_path = f'../../data/raw_data/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten.csv'
    csv_path1 = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_removed_preload.csv'
    # csv_path1 = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_labled.csv'
    csv_path2 = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
    df = pd.read_csv(csv_path, delimiter=',')
    number_labled_pairs = len(pd.read_csv(csv_path1, delimiter=','))
    df_labled = create_plagiate_table(
        df, semester, ha, task, prog_language, number_labled_pairs, csv_path2)
    return df_labled


def create_plagiate_table_from_df(df, semester, ha, prog_language, plagiate_functions=[remove_comments, variable_renamer]):
    save_path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
    number_labled_pairs = len(df)
    df = df.drop_duplicates(subset=['surname1'])
    number_of_valid_codes = len(df)
    print("number_of_valid_codes: " + str(number_of_valid_codes))
    print("number_labled_pairs: " + str(number_labled_pairs))
    plagiate_per_solution = int(np.floor(number_labled_pairs/number_of_valid_codes))
    print("plagiate_per_solution: " + str(plagiate_per_solution))
    # drop unnecessary columns
    df = df.drop(columns=['hand_labled', 'Unnamed: 0', 'surname2', 'lastname2'])
    df = df.reindex(df.index.repeat(plagiate_per_solution))
    df['code2'] = df['code1'].apply(func[0])
    for func in plagiate_functions[1:]:
        df['code2'] = df['code2'].apply(func)
    df['label'] = 1
    df.to_csv(save_path)
    return df


def split_data_from_labled(semester, ha, prog_language):
    df_labled = pd.read_csv(f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_labled.csv', delimiter=',')
    df_labled = remove_given_code_from_df(df_labled)
    df_plagiate = create_plagiate_table_from_df(df_labled, semester, ha, prog_language)
    return df_plagiate
    

def open_csvs(semester, ha, prog_language):
    original_csv_path = f'../../data/raw_data/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten.csv'
    labled_csv_path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_removed_preload.csv'
    plagiate_csv_path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
    df_original = pd.read_csv(original_csv_path, delimiter=',')
    df_labled = pd.read_csv(labled_csv_path, delimiter=',')
    return df_original, df_labled, plagiate_csv_path


In [3]:
semester='SoSe22'
ha='9'
task='Antwort 9'
prog_language='C'
# path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_plagiate.csv'
path = f'../../data/labled/PPR [{semester}]-{ha}. Hausaufgabe - Pflichttest {prog_language}-Antworten_labled.csv'

In [4]:
def add_copy_and_paste_plagiate(df):
    if not df[df['lastname1'] == df['lastname2']].empty:
        print('Dataframe already contains copy and paste plagiate')
        return df
    if 'Unnamed: 0' in df.columns:
        df = df.drop(columns=['Unnamed: 0'])
    df_copy_and_paste = df.drop_duplicates(subset=['surname1']).copy()
    number_of_valid_codes = len(df_copy_and_paste)
    print("number_of_valid_codes: " + str(number_of_valid_codes))
    df_copy_and_paste['code2'] = df_copy_and_paste['code1']
    df_copy_and_paste[['surname2', 'lastname2']] = df_copy_and_paste[['surname1', 'lastname1']]
    df_copy_and_paste['hand_labled'] = 1
    df_copy_and_paste['label'] = 1
    df = pd.concat([df, df_copy_and_paste])
    return df

# df_plagiate = split_data_from_labled(semester, ha, prog_language)
# df = pd.read_csv(path, delimiter=',')
# df = df.drop(columns=['Unnamed: 0'])
# print("len df: " + str(len(df)))
# df = add_copy_and_paste_plagiate(df)
# print("len df: " + str(len(df)))

# df['label'] = df['label'].apply(lambda x: 1 if x > 0.9 else 0)
# df.to_csv(path)



In [5]:
def add_copy_and_paste_plagiate_routine(df, filename, directory='../../data/labled'):
    if not df[df['lastname1'] == df['lastname2']].empty:
        print(f'Dataframe {filename} already contains copy and paste plagiate')
        return len(df[df['lastname1'] == df['lastname2']]), len(df)
    else:
        print(f'Dataframe {filename} does not contain copy and paste plagiate')
        df = add_copy_and_paste_plagiate(df)
        df.to_csv(f'{directory}/{filename}')
        

# open all csvs in labled folder and check if they contain copy and paste plagiate
import os
# count_copy_and_paste_plagiate = 0
# count = 0
# for filename in os.listdir('../../data/labled'):
#     if filename.endswith("labled.csv"):
#         print(filename)
#         df = pd.read_csv(f'../../data/labled/{filename}', delimiter=',')
#         a, b = add_copy_and_paste_plagiate_routine(df, filename)
#         count_copy_and_paste_plagiate, count = count_copy_and_paste_plagiate + a, count + b
#     else:
#         continue
# # plagiate = pd.read_csv('../../data/labled/PPR [SoSe21]-9. Hausaufgabe - Pflichttest C-Antworten_plagiate.csv', delimiter=',')
# print(f'count_copy_and_paste_plagiate: {count_copy_and_paste_plagiate}')
# print(f'count: {count}')
# print(f'count - count_copy_and_paste_plagiate: {count - count_copy_and_paste_plagiate}')

In [6]:
def remove_specific_students(df_labled, names):
    for name in names:
        df_labled = df_labled.drop(df_labled[(df_labled['lastname1'] == name) | (df_labled['lastname2'] == name)].index)
    return df_labled


# df_labled = pd.read_csv(path)
# try:
#     df_labled = df_labled.drop(columns=['Unnamed: 0'])
# except KeyError:
#     pass
# print(df_labled.columns)
# names = ['Lehmann', 'Hengst', 'Hilger', 'Krause']
# df_labled = remove_specific_students(df_labled, names)

# print(df_labled['label'].mean())
# df_labled['label'].hist()
# df_labled['hand_labled'].hist()

# df_labled.to_csv(path, index=False)
# vorgabe entfernen, auto labeln lassen und dann plagiate erstellen

In [7]:
import re
def striphtml(data): 
    return re.sub('<.*?>', ' ', str(data)) 

def stripunc(data): 
    return re.sub('[^A-Za-z%\._\[\]]+', ' ', str(data), flags=re.MULTILINE|re.DOTALL) 

In [8]:
# # for prasentation to show example strip
# read_path='/home/erik/TU/ni/plagiarism_detection_tool/data/labled/handmade_plagiate_0_0.c'
# write_path='/home/erik/TU/ni/plagiarism_detection_tool/data/labled/handmade_plagiate_0_3.c'
# with open(read_path, 'r') as file:
#     code = file.read()
#     code = striphtml(stripunc(code))
#     with open(write_path, 'w') as file:
#         file.write(code)

## Regex's to automatically refactor c code

In [62]:
def swap_loops_type(given_string):
    matches = [x.group() for x in re.finditer(r'for|while', given_string)]
    last_index = 0
    for match in matches:
        if match == 'for':
            rest_string, new_index = for_to_while(given_string[last_index:])
            given_string = given_string[:last_index] + rest_string
        elif match == 'while':
            rest_string, new_index = while_to_for(given_string[last_index:])
            given_string = given_string[:last_index] + rest_string
        last_index += new_index
    return given_string

# call this function without opening curly braces in substring
def idx_closing_brace(given_substring):
    sum_braces = 1
    for i, char in enumerate(given_substring):
        if char == '{':
            sum_braces += 1
        elif char == '}':
            sum_braces -= 1
        if sum_braces == 0:
            return i
    return -1


def for_to_while(given_string):
    pattern = '\n*(\s*)for\s*\((.*?;)(.*?)(;.*?)\)(\s*{+)'
    #TODO removed oneliner; does not work with missing braces
    # find first match of pattern in string
    match = re.search(pattern, given_string)
    if match is None:
        return given_string, 0
    # assign groups to variables
    index, spaces, init, cond, incr, open_brace = match.end(0), match.group(1), match.group(2), match.group(3), match.group(4), match.group(5)
    pattern_tmp = 'for\s*\(' + re.escape(''.join([init, cond, incr])) + '\)(\s*{+)*'
    cond = cond.strip()
    # did not work if condition is i.e (tmp != '\0')
    cond = cond.encode('unicode_escape').decode()
    # remove ';' at the beginning and add it at the end of the of the third part
    incr = incr[1:].strip()
    incr = incr + ';'
    idx_brace = idx_closing_brace(given_string[index+1:])
    init_line = init + '\n' + spaces if init != ';' else ''
    if idx_brace == -1:
        # insert necessary braces
        if incr != ';':
            given_string = given_string[:index] + re.sub(';', ';\n\t' + spaces + incr + '\n' + spaces + '}', given_string[index:], 1)
        given_string = re.sub(pattern_tmp, init_line + 'while (' + cond + '){', given_string, 1)
    else:
        # add missing offset from index+1
        idx_brace += index+1
        if incr != ';':
            given_string = given_string[:idx_brace] + re.sub('}', '\t' + incr + '\n' + spaces + '}', given_string[idx_brace:], 1)
        given_string = re.sub(pattern_tmp, init_line + 'while (' + cond + ')' + open_brace, given_string, 1)
    # return new index of closing brace after while condition
    index = given_string.find('while (' + cond + ')') + len('while (' + cond + ')')
    return given_string, index


# does not work with do while loops
def while_to_for(given_string):
    # TODO add check for previous line and try to put it in the init part (\S* *\S*.*;)\s*
    pattern = 'while\s*\((.*?)\)(\s*{)(?:(?:.|\n)*?)\s*(.*;)\s*}'
    match = re.search(pattern, given_string) 
    if match is None:
        return given_string, 0
        # skip if there is no closing brace
        pattern_without_braces = 'while\s*\((.*?)\)(\s*)(?:(?:.|\n)*?)\s*(.*;)\s*'
        match = re.search(pattern_without_braces, given_string)
        if match is None:
            return given_string, 0
    index, cond, open_brace, incr = match.end(2), match.group(1), match.group(2), match.group(3)
    pattern_tmp = 'while\s*\(' + re.escape(cond) + '\)(\s*{+)*'
    cond = cond.strip()
    # did not work if condition is i.e (tmp != '\0')
    cond = cond.encode('unicode_escape').decode()
    if incr != '':
        given_string = re.sub(';\s*' + re.escape(incr), ';', given_string, 1)
        incr = ' ' + incr[:-1].strip()
    given_string = re.sub(pattern_tmp, 'for (; ' +  cond + ';' + incr + ')' + open_brace, given_string, 1)
    # return new index of closing brace after for condition
    index = given_string.find('for (; ' + cond + ';' + incr + ')') + len('for (; ' + cond + ';' + incr + ')')
    return given_string, index

In [10]:
def mirror_logic(logic):
    match logic:
        case '<':
            return '>'
        case '>':
            return '<'
        case '<=':
            return '>='
        case '>=':
            return '<='
        case _:
            return logic

# swap variables in logic equations like a <= b to b >= a
def mirrow_simple_logic_equations(given_string):
    pattern = '\s*([^;()]+?)\s*(!=|==|\|\||&&|<=|>=|<|>)\s*([^;()]+)'
    indexes = [(x.start(1),x.end(0)) for x in re.finditer(pattern, given_string)]
    matches = re.findall(pattern, given_string)
    for (start, end), (a, logic, b) in zip(indexes, matches):
        #TODO also temporary remove i.e. c = a < b; to avoid not working matching
        if 'include' in a or 'define' in a: # is a workaround for the problem that the pattern matches also in #include <stdio.h>
            continue
        #TODO escape a and b?
        if a == '\0' or b == '\0':
            continue
        given_string = given_string[:start] + f'{b.strip()} {mirror_logic(logic)} {a.strip()}' + given_string[end:]
    return given_string

def mirrow_nested_logic_equations(given_string):
    pass #([\S()]+)\s*(!=|==|\|\||&&|<=|>=|<|>)\s*([\s()]+)


In [11]:
def invert_logic(logic):
    match logic:
        case '<=':
            return '>'
        case '>=':
            return '<'
        case '<':
            return '>='
        case '>':
            return '<='
        case '==':
            return '!='
        case '!=':
            return '=='
        case '&&':
            return '||'
        case '||':
            return '&&'
        case _:
            return logic

def demorgan_on_logic_equations(given_string):
    pattern = '\(((?:[;()\S ]+?)\s*(!=|==|\|\||&&|<=|>=|<|>)\s*(?:[()\S ;]+))\)'
    indexes = [(x.start(0),x.end(0)) for x in re.finditer(pattern, given_string)]
    matches = re.findall(pattern, given_string)
    for (start, end), (match, logic) in zip(indexes, matches):
        given_string = given_string[:start] + re.sub(re.escape(match), f'!({match.replace(logic, invert_logic(logic))})', given_string[start:], count=1) 
    return given_string


In [12]:
def all_idx_braces(given_string):
    idx_braces = []
    for i, char in enumerate(given_string):
        if char == '{':
            idx_braces.append(i)
        if char == '}':
            idx_braces.append(i)
    return idx_braces

def swap(c, i, j):
    c = list(c)
    c[j.start:j.stop], c[j.stop:i.start], c[i.start:i.stop] = c[i.start:i.stop], c[j.stop:i.start], c[j.start:j.stop]
    return ''.join(c)

def swap_functions(given_string):
    functions = [(x.start(0), x.group(1), x.end(0)) for x in re.finditer("\S+ +([a-zA-Z_][a-zA-Z0-9_]+)\s*\(.*\s*{", given_string) if x.group(1) != "if" and x.group(1) != "for" and x.group(1) != "main"]
    idx_list = []
    if len(functions) > 1:
        for start_idx, name, opening_brace_idx in functions:
            closing_brace_idx = idx_closing_brace(given_string[opening_brace_idx:]) + opening_brace_idx + 1
            idx_list.append(slice(start_idx, closing_brace_idx))
        return swap(given_string, idx_list[0], idx_list[1])
    return given_string



#TODO function that swaps spelling from array access to pointer access and vice versa
# i.e. a[0] to *a and *a to a[0]

## Testcases

In [61]:
import unittest

# TODO rewrite test functions to test subclasses
# TODO tests all edge cases, because students dont write good code
# import pdb; pdb.set_trace() # debug
class Test(unittest.TestCase):
    given_string = 'for (int i = 0; i < 10; i++){\n\tprintf("%d", i);\n}\nint abc = 0;\nwhile (abc < 10){\n\tprintf("%d", abc);\n\tabc++;\n}'
    given_string2 = 'for (int i = 0; i < 10; i++){\n\tfor (int j = 0; j < 10; j++){\n\t\tint alpha=0;\n\t\twhile(alpha<3){\n\t\t\tprintf("%d", i);\n\t\t\talpha++;\n\t\t}\n\t}\n}'
    given_string3 = 'for (int i = 0; i < 10; i++){\n\tprintf("%d", i);\n}'

    def test_remove_comments(self):
        self.assertEqual(remove_comments(self.given_string), self.given_string)
        comment_string = '// Funktion die bis 100 zählt\nfor (int i = 0; i < 10; i++){\n\t/* zweite Schleife\n\t   damit wird 10*10 gerechnet */\n\tfor (int j = 0; j < 10; j++){\n\t\tprintf("%d", i);\t// das ist ein printf\n\t}\n}'
        self.assertEqual(remove_comments(comment_string), '\nfor (int i = 0; i < 10; i++){\n\tfor (int j = 0; j < 10; j++){\n\t\tprintf("%d", i);\n\t}\n}')
        with open('/home/erik/TU/ni/plagiarism_detection_tool/data/labled/handmade_plagiate_6_0.c', 'r') as file:
            string2 = file.read()
        with open('/home/erik/TU/ni/plagiarism_detection_tool/data/labled/handmade_plagiate_6_0_removed_comments.c', 'r') as file:
            string2_solution = file.read()
        self.assertEqual(remove_comments(string2), string2_solution)

    def test_swap_loops_type(self):
        self.assertEqual(swap_loops_type(self.given_string), 'int i = 0;\nwhile (i < 10){\n\tprintf("%d", i);\n\ti++;\n}\nint abc = 0;\nfor (; abc < 10; abc++){\n\tprintf("%d", abc);\n}')
        self.assertEqual(swap_loops_type(self.given_string2), 'int i = 0;\nwhile (i < 10){\n\tint j = 0;\n\twhile (j < 10){\n\t\tint alpha=0;\n\t\tfor (; alpha<3; alpha++){\n\t\t\tprintf("%d", i);\n\t\t}\n\t\tj++;\n\t}\n\ti++;\n}')
        self.assertEqual(swap_loops_type(self.given_string3), 'int i = 0;\nwhile (i < 10){\n\tprintf("%d", i);\n\ti++;\n}')

    def test_for_to_while(self):
        # doesnt work with nested for loops *check*
        self.assertEqual(for_to_while(self.given_string), ('int i = 0;\nwhile (i < 10){\n\tprintf("%d", i);\n\ti++;\n}\nint abc = 0;\nwhile (abc < 10){\n\tprintf("%d", abc);\n\tabc++;\n}', 25))
        # self.assertEqual(for_to_while(self.given_string2), 'int i = 0;\nwhile (i < 10){\n\tint j = 0;\n\twhile (j < 10){\n\t\tint alpha=0;\n\t\twhile(alpha<3){\n\t\t\tprintf("%d", i);\n\t\t\talpha++;\n\t\t}\n\t\tj++;\n\t}\n\ti++;\n}')
        one_liner = 'for (int i = 0; i < 10; i++)\n\tprintf("%d", i);'
        expected = 'int i = 0;\nwhile (i < 10){\n\tprintf("%d", i);\n\ti++;\n}'
        # self.assertEqual(for_to_while(one_liner), (expected, 25))
        self.assertEqual(for_to_while('for (int i = 0; i < 10, i++)\n\tprintf("%d", i);'), ('for (int i = 0; i < 10, i++)\n\tprintf("%d", i);', 0))
        # right amount of spaces in inserted code *check*
        # must work independent if there is space between ) and {
        one_liner = 'int i = 0;\nfor (;i < 10;) {\n\tprintf("%d", i);\n\ti++;\n}'
        expected = 'int i = 0;\nwhile (i < 10) {\n\tprintf("%d", i);\n\ti++;\n}'
        self.assertEqual(for_to_while(one_liner), (expected, 25))
        # test if while is correctly indented *check*
        one_liner = '\tint i = 0;\n\tfor (;i < 10;) {\n\t\tprintf("%d", i);\n\t\ti++;\n\t}'
        expected = '\tint i = 0;\n\twhile (i < 10) {\n\t\tprintf("%d", i);\n\t\ti++;\n\t}'
        self.assertEqual(for_to_while(one_liner), (expected, 27))
        # test mit \0 *check*
        self.assertEqual(for_to_while('for(char* p = argv[j]; (*p) != "\\0"; p++){\n\tkollektion[sammelzaehler] = *p;\n}'), ('char* p = argv[j];\nwhile ((*p) != "\\0"){\n\tkollektion[sammelzaehler] = *p;\n\tp++;\n}', 20))

    def test_while_to_for(self):
        # would be nice if increment in while is found and inserted in for loop *check*
        self.assertEqual(while_to_for(self.given_string), ('for (int i = 0; i < 10; i++){\n\tprintf("%d", i);\n}\nint abc = 0;\nfor (; abc < 10; abc++){\n\tprintf("%d", abc);\n}',86))
        self.assertEqual(while_to_for(self.given_string2), ('for (int i = 0; i < 10; i++){\n\tfor (int j = 0; j < 10; j++){\n\t\tint alpha=0;\n\t\tfor (; alpha<3; alpha++){\n\t\t\tprintf("%d", i);\n\t\t}\n\t}\n}',102))
        two_liner = '\tint i = 0;\n\twhile (i < 10)    {\n\t\tprintf("%d", i);\n\t\ti++;\n\t}'
        expected = '\tint i = 0;\n\tfor (; i < 10; i++)    {\n\t\tprintf("%d", i);\n\t}'
        self.assertEqual(while_to_for(two_liner), (expected, 32))
        two_liner = '  int abc = 0;\n  while (abc < 10){\n    printf("%d", abc);\n    --abc;\n  }'
        expected = '  int abc = 0;\n  for (; abc < 10; --abc){\n    printf("%d", abc);\n  }'
        self.assertEqual(while_to_for(two_liner), (expected, 40))
        self.assertEqual(while_to_for('printf("Hello World\n");'), ('printf("Hello World\n");', 0))
        self.assertEqual(while_to_for('char* p = argv[j];\nwhile(*p != "\\0"){\n\tkollektion[sammelzaehler] = *p;\n\tp++;\n}'),('char* p = argv[j];\nfor (; *p != "\\0"; p++){\n\tkollektion[sammelzaehler] = *p;\n}', 23))
        # self.assertEqual(while_to_for(' while((product%div)!=0)\n   div--;'), (' for (; (product%div)!=0; div--);', 0))

    def idx_opening_brace(self,given_string):
        for i, char in enumerate(given_string):
            if char == '{':
                return i
        return -1

    def all_idx_braces(self,given_string):
        idx_braces = []
        for i, char in enumerate(given_string):
            if char == '{':
                idx_braces.append(i)
            if char == '}':
                idx_braces.append(i)
        return idx_braces

    def test_index_of_braces(self):
        self.assertEqual(idx_closing_brace(self.given_string3), -1)
        index = self.idx_opening_brace(self.given_string2)
        self.assertEqual(idx_closing_brace(self.given_string2[index+1:])+index+1, 133)

    def test_mirror_simple_logic_equation(self):
        self.assertEqual(mirrow_simple_logic_equations("#include <stdio.h>\nif( *p!='x' )"), "#include <stdio.h>\nif( 'x' != *p)")
        self.assertEqual(mirrow_simple_logic_equations("if( *p || 'x' )"), "if( 'x' || *p)")
        self.assertEqual(mirrow_simple_logic_equations("if( *p && 'x' )"), "if( 'x' && *p)")
        self.assertEqual(mirrow_simple_logic_equations("if( *p<='x' )"), "if( 'x' >= *p)")
        self.assertEqual(mirrow_simple_logic_equations("if( *p>='x' )"), "if( 'x' <= *p)")
        self.assertEqual(mirrow_simple_logic_equations("if( *p>'x' )"), "if( 'x' < *p)")
        self.assertEqual(mirrow_simple_logic_equations("if( *p<'x' )"), "if( 'x' > *p)")
        self.assertEqual(mirrow_simple_logic_equations("if( *p=='x' )"), "if( 'x' == *p)")
        self.assertEqual(mirrow_simple_logic_equations("if(*p<'x')"), "if('x' > *p)")
        self.assertEqual(mirrow_simple_logic_equations("if(*p=='x')"), "if('x' == *p)")
        self.assertEqual(mirrow_simple_logic_equations("if( *p < 'x')"), "if( 'x' > *p)")
        self.assertEqual(mirrow_simple_logic_equations("if( *p == 'x')"), "if( 'x' == *p)")
        self.assertEqual(mirrow_simple_logic_equations("if(1 + *p > 'x' + 3)"), "if('x' + 3 < 1 + *p)")
        self.assertEqual(mirrow_simple_logic_equations("      if((*p != 'x')&&(*p || 'y'))"), "      if(('x' != *p)&&('y' || *p))")

    def test_demorgan_on_logic_equations(self):
        self.assertEqual(demorgan_on_logic_equations("if( *p!='x' )"), "if(!( *p=='x' ))")
        self.assertEqual(demorgan_on_logic_equations("if( *p || 'x' )"), "if(!( *p && 'x' ))")
        self.assertEqual(demorgan_on_logic_equations("if( *p && 'x' )"), "if(!( *p || 'x' ))")
        self.assertEqual(demorgan_on_logic_equations("if( *p<='x' )"), "if(!( *p>'x' ))")
        self.assertEqual(demorgan_on_logic_equations("if( *p>='x' )"), "if(!( *p<'x' ))")
        self.assertEqual(demorgan_on_logic_equations("if( *p>'x' )"), "if(!( *p<='x' ))")
        self.assertEqual(demorgan_on_logic_equations("if( *p<'x' )"), "if(!( *p>='x' ))")
        self.assertEqual(demorgan_on_logic_equations("if( *p=='x' )"), "if(!( *p!='x' ))")
        self.assertEqual(demorgan_on_logic_equations("if(*p<'x')"), "if(!(*p>='x'))")
        self.assertEqual(demorgan_on_logic_equations("if(*p=='x')"), "if(!(*p!='x'))")
        self.assertEqual(demorgan_on_logic_equations("if( *p < 'x')"), "if(!( *p >= 'x'))")
        self.assertEqual(demorgan_on_logic_equations("if( *p == 'x' )"), "if(!( *p != 'x' ))")
        self.assertEqual(demorgan_on_logic_equations("if(1 + *p > 'x' + 3)"), "if(!(1 + *p <= 'x' + 3))")
        self.assertEqual(demorgan_on_logic_equations("while(i < sammelzaehler)"), "while(!(i >= sammelzaehler))")
        #TODO nested demorgan
        # self.assertEqual(demorgan_on_logic_equations("while('a' <= a && a <= 'z')"), "while(!('a' > a || a > 'z'))")
        # self.assertEqual(demorgan_on_logic_equations("while('a' <= a && a <= 'z' && a < 'y')"), "while(!('a' > a || a > 'z' || a >= 'y'))")
        # self.assertEqual(demorgan_on_logic_equations("      if((*p != 'x')&&(*p || 'y'))"), "      if(('x' == *p)||('y' && *p))")
        #TODO
        # self.assertEqual(demorgan_on_logic_equations("for(; i < sammelzaehler ; i += 3)"), "for(; !(i >= sammelzaehler) ; i += 3)")

    def test_swap_functions(self):
        with open('/home/erik/TU/ni/plagiarism_detection_tool/data/labled/handmade_plagiate_3_0.c', 'r') as file:
            string1 = file.read()
        with open('/home/erik/TU/ni/plagiarism_detection_tool/data/labled/handmade_plagiate_3_0_swaped_functions.c', 'r') as file:
            string1_solution = file.read()
        self.assertEqual(swap_functions(string1), string1_solution)
        with open('/home/erik/TU/ni/plagiarism_detection_tool/data/labled/handmade_plagiate_6_0.c', 'r') as file:
            string2 = file.read()
        with open('/home/erik/TU/ni/plagiarism_detection_tool/data/labled/handmade_plagiate_6_0_removed_comments_and_swaped_functions.c', 'r') as file:
            string2_solution = file.read()
        self.assertEqual(swap_functions(remove_comments(string2)), string2_solution)
        self.assertEqual(swap_functions(string2_solution), remove_comments(string2))

    def test_swap(self):
        string = 'My name is Erik and my friend is called Max'
        string_solution = 'My name is Max and my friend is called Erik'
        self.assertEqual(swap(string, slice(11,15), slice(40,len(string))), string_solution)
        string = 'My name is Max and my friend is called Erik'
        string_solution = 'My name is Erik and my friend is called Max'
        self.assertEqual(swap(string, slice(11,14), slice(39,len(string))), string_solution)


unittest.main(argv=[''], verbosity=2, exit=False)

test_demorgan_on_logic_equations (__main__.Test) ... ok
test_for_to_while (__main__.Test) ... ok
test_index_of_braces (__main__.Test) ... ok
test_mirror_simple_logic_equation (__main__.Test) ... ok
test_remove_comments (__main__.Test) ... ok
test_swap (__main__.Test) ... ok
test_swap_functions (__main__.Test) ... ok
test_swap_loops_type (__main__.Test) ... ok
test_while_to_for (__main__.Test) ... ok

----------------------------------------------------------------------
Ran 9 tests in 0.012s

OK


# Use functions to refactor code

In [87]:
from itertools import cycle

path_plagiate = '/home/erik/TU/ni/plagiarism_detection_tool/data/labled/2023_01_02/all_plagiate_data.csv'
path_labled = '/home/erik/TU/ni/plagiarism_detection_tool/data/labled/2023_01_02/all_labled_data.csv'
df_labled = pd.read_csv(path_labled, sep=',')


def apply_refactoring_functions(df, semester, ha, prog_language, plagiate_functions=[variable_renamer, swap_loops_type, swap_functions, demorgan_on_logic_equations, mirrow_simple_logic_equations]):
    number_labled_pairs = len(df)
    # drop duplicates with same surname1, semester, ha, prog_lang, task
    df_refactor = df.drop_duplicates(subset=['surname1', 'semester', 'ha', 'prog_lang', 'task'])
    number_of_valid_codes = len(df_refactor)
    print("number_of_valid_codes: " + str(number_of_valid_codes))
    print("number_labled_pairs: " + str(number_labled_pairs))
    df_plagiate = pd.read_csv(path_plagiate, sep=',')
    df_plagiate = df_plagiate.drop(columns=['Unnamed: 0'])
    df_plagiate['type'] = 'c&p'
    # calculate how many plagiate per solution without copy and paste plagiate
    plagiate_per_solution = int(np.floor((number_labled_pairs-len(df_plagiate))/number_of_valid_codes))
    print("plagiate_per_solution: " + str(plagiate_per_solution))
    # drop unnecessary columns
    df_refactor = df_refactor.drop(columns=['hand_labled', 'Unnamed: 0', 'surname2', 'lastname2'])
    df_refactor = remove_given_code_from_df(df_refactor)
    df_refactor['code2'] = df_refactor['code1'].apply(remove_comments)
    df_refactor = df_refactor.reindex(df_refactor.index.repeat(plagiate_per_solution))
    df_refactor = df_refactor.reset_index(drop=True)
    # loop through rows in df_refactor and repeated list of plagiate_functions and apply them to code2
    for (index, row), plagiate_function in zip(df_refactor.iterrows(), cycle(plagiate_functions)):
        df_refactor.at[index, 'code2'] = plagiate_function(df_refactor.at[index, 'code2'])
        df_refactor.at[index, 'type'] = f'{plagiate_function.__name__}'
    df_refactor['label'] = 1
    df_refactor = pd.concat([df_refactor, df_plagiate])
    # df_refactor.to_csv(path_plagiate)
    return df_refactor


def create_plagiate_from_labled(path):
    df_labled = pd.read_csv(path, delimiter=',')
    df_plagiate = apply_refactoring_functions(df_labled, semester, ha, prog_language)
    return df_plagiate

data = create_plagiate_from_labled(path=path_labled)



number_of_valid_codes: 121
number_labled_pairs: 1482
plagiate_per_solution: 11


In [91]:
# drop surname2, lastname2, hand_labled
data = data.drop(columns=['surname2', 'lastname2', 'hand_labled'])

In [97]:
import csv
df_labled.to_csv(path_or_buf=path_labled, sep=',')
data.to_csv(path_or_buf=path_plagiate, sep=',', quoting=csv.QUOTE_ALL, escapechar='\\')

In [86]:
# df_plagiate = pd.read_csv(path_plagiate, sep=',')
# # drop all rows with have not the same surname1 and surname2
# df_plagiate = df_plagiate[df_plagiate['surname1'] == df_plagiate['surname2']]
# # save df
# df_plagiate.to_csv(path_plagiate)

In [ ]:
# tmp = df_labled.drop_duplicates(subset=['surname1', 'semester', 'ha', 'prog_lang', 'task'])
# tmp['code2'] = tmp['code1']
# tmp['surname2'] = tmp['surname1']
# tmp['lastname2'] = tmp['lastname1']
# tmp['label'] = 1
# # drop unnecessary columns
# tmp = tmp.drop(columns=['Unnamed: 0'])
# tmp.to_csv(path_plagiate)